# Simple Model

- We don't have returns after 2022 Jan. 
- We have data until 2022 Sept.

- For model training:
    - Testing period --> 2021 June - 2022 Jan
    - Train period: 2017 June - 2021 June (3 years)
    One stock per line. Aggregate all data into one row

- For Inference:
    - Infer for periods between 2019 Sept. - 2022 Sept.
    - This will give us the returns for 2022 Sept. until 2023 March. 
    - Choose the top 15 stocks



In [1]:
import pandas as pd
import numpy as np
import Preprocessing as pr
import datetime as dt

In [2]:
df = pr.get_data()

/Users/kaancaylan/Desktop/HEC/DataProjectQuinten-StockPrediction/quinten-stock-prediction/Preprocessing.py:8: DtypeWarning: Columns (8,17,26,33,34,66,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,252,253,255,257,259,261,263,265,267,269,270,271,272,273,274,275,276,277,278,279,280,284,285,286,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";", parse_dates=date_cols).iloc[:, 1:]


In [3]:
df[df.year==2022].groupby("quarter").apply(lambda x: len(x.index.get_level_values(1).unique()))

quarter
1    1407
3    1407
6    1407
9    1407
dtype: int64

In [4]:
df[df.year==2022].close.dropna()

date        symbol  
2022-01-31  0O9.F        11.952000
            0QF.F       148.268000
            217A.F        7.990000
            22UA.F      156.159998
            2CRSI.PA      4.443000
                           ...    
            XIL.PA       38.820000
            XIOR.BR      48.390000
            YATRA.AS      3.300000
            YK6B.F       15.350000
            ZEL.NZ        3.598000
Name: close, Length: 1387, dtype: float64

# Model: XGBoost 

__Only keep stocks that are trading in Euros
currency_by_stock = df["reportedCurrency"].groupby("symbol").apply(lambda x:x.dropna()[0] if len(x.dropna()) > 0 else np.nan)
euro_stocks = currency_by_stock[currency_by_stock=="EUR"].index
idx = pd.IndexSlice[:, euro_stocks]
df = df.loc[idx, :]

In [5]:
# Set train and validation dates
train_dates = {"start": dt.datetime(2017, 6, 30), "end": dt.datetime(2021, 6, 30)}
val_dates = {"start": dt.datetime(2021, 6, 30), "end": dt.datetime(2022, 1, 31)}

X_train = df.loc[train_dates["start"]: train_dates["end"]]
X_train = X_train.unstack(level=0).drop("year", axis=1)
X_train



quarter                                                         \
date     2017-06-30 2017-09-30 2018-01-31 2018-03-31 2018-06-30 2018-09-30   
symbol                                                                       
0O9.F             6          9          1          3          6          9   
0QF.F             6          9          1          3          6          9   
217A.F            6          9          1          3          6          9   
22UA.F            6          9          1          3          6          9   
2CRSI.PA          6          9          1          3          6          9   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR           6          9          1          3          6          9   
YATRA.AS          6          9          1          3          6          9   
YK6B.F            6          9          1          3          6          9   
ZEL.NZ            6          9          1          3          6          9   
ZEN.BR            6          9          1          3          6          9   

                                                      ...     return  \
date     2019-01-31 2019-03-31 2019-06-30 2019-09-30  ... 2019-03-31   
symbol                                                ...              
0O9.F             1          3          6          9  ...   0.108493   
0QF.F             1          3          6          9  ...   0.281364   
217A.F            1          3          6          9  ...  -0.077690   
22UA.F            1          3          6          9  ...        NaN   
2CRSI.PA          1          3          6          9  ...   0.010842   
...             ...        ...        ...        ...  ...        ...   
XIOR.BR           1          3          6          9  ...   0.082149   
YATRA.AS          1          3          6          9  ...  -0.017045   
YK6B.F            1          3          6          9  ...        NaN   
ZEL.NZ            1          3          6          9  ...   0.031841   
ZEN.BR            1          3          6          9  ...   0.096774   

                                                                            \
date     2019-06-30 2019-09-30 2020-01-31 2020-03-31 2020-06-30 2020-09-30   
symbol                                                                       
0O9.F     -0.251957   0.420561   0.038158  -0.205070   1.377232   0.080483   
0QF.F     -0.276008   0.143268   0.432857   0.421137   0.880384   0.121367   
217A.F    -0.058054   0.034743   0.242336   0.471210   0.773163  -0.127928   
22UA.F          NaN        NaN        NaN   0.792782   0.159442  -0.006257   
2CRSI.PA  -0.034241  -0.475438  -0.098941  -0.498644   0.702118  -0.161769   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR    0.064371   0.063473   0.135666  -0.270473   0.230433   0.156309   
YATRA.AS   0.158382  -0.051896   0.052632   0.030000  -0.076699  -0.100946   
YK6B.F          NaN        NaN        NaN        NaN        NaN        NaN   
ZEL.NZ     0.007715  -0.118660  -0.189287  -0.350446  -0.063918   0.005140   
ZEN.BR    -0.071765   0.001267   0.228354  -0.094188   0.116041   0.033537   

                                           
date     2021-01-31 2021-03-31 2021-06-30  
symbol                                     
0O9.F      1.379888  -0.230569  -0.085763  
0QF.F      1.177801  -0.179831   0.822632  
217A.F     0.657025  -0.150561  -0.313028  
22UA.F     0.518609  -0.002897   1.111287  
2CRSI.PA   0.490524   0.077559  -0.172468  
...             ...        ...        ...  
XIOR.BR   -0.074919  -0.106025   0.114661  
YATRA.AS   0.111111  -0.025000  -0.016464  
YK6B.F          NaN   0.099440   0.150318  
ZEL.NZ     0.105917  -0.058785  -0.045614  
ZEN.BR     0.134234  -0.013043  -0.051057  

[1407 rows x 4845 columns]

In [6]:
# y_train = 
test_per_rets = df.loc[val_dates["start"]: val_dates["end"], "return"]
test_per_rets

date        symbol  
2021-06-30  0O9.F      -0.085763
            0QF.F       0.822632
            217A.F     -0.313028
            22UA.F      1.111287
            2CRSI.PA   -0.172468
                          ...   
2022-01-31  XIOR.BR    -0.023805
            YATRA.AS    0.000000
            YK6B.F     -0.085221
            ZEL.NZ      0.055132
            ZEN.BR      0.000000
Name: return, Length: 4221, dtype: float64

In [7]:
# drop stocks with missing returns for this period
to_drop = test_per_rets[test_per_rets.isna()].index.get_level_values("symbol").unique()

# drop stocks that traded under 1$ in the period
under_1 = X_train['close'].groupby(level="symbol").apply(lambda x: (x<1).any(axis=1)).droplevel(1)
to_drop = to_drop.union(under_1[under_1].index).unique()

X_train = X_train.drop(to_drop)
y_train = test_per_rets.drop(to_drop, level=1)

In [8]:
print(np.setdiff1d(X_train.index, y_train.index.get_level_values(1)))
print(np.setdiff1d(y_train.index.get_level_values(1), X_train.index))

[]
[]


In [9]:
y_train = y_train.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])

assert all(X_train.index == y_train.index)

In [10]:
# Last checks and drops on columns
X_train = X_train.select_dtypes(exclude=["object", "datetime"])

In [11]:
X_train.dtypes.value_counts()


float64    4743
int64        34
dtype: int64

In [12]:
from xgboost import XGBRegressor

/Users/kaancaylan/.pyenv/versions/3.9.6/envs/myenv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [13]:
X_train.shape

(1141, 4777)

In [14]:
model = XGBRegressor()
model.fit(X_train.to_numpy(), y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=1,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [18]:
model.score(X_train, y_train)

0.9994187457919423

# Predicting Returns

In [40]:
infer_dates = {"start": dt.datetime(2018, 9, 30), "end": dt.datetime(2022, 9, 30)}

X_test = df.loc[infer_dates["start"]: infer_dates["end"]]
X_test = X_test.unstack(level=0).drop("year", axis=1)
X_test

quarter                                                         \
date     2018-09-30 2019-01-31 2019-03-31 2019-06-30 2019-09-30 2020-01-31   
symbol                                                                       
0O9.F             9          1          3          6          9          1   
0QF.F             9          1          3          6          9          1   
217A.F            9          1          3          6          9          1   
22UA.F            9          1          3          6          9          1   
2CRSI.PA          9          1          3          6          9          1   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR           9          1          3          6          9          1   
YATRA.AS          9          1          3          6          9          1   
YK6B.F            9          1          3          6          9          1   
ZEL.NZ            9          1          3          6          9          1   
ZEN.BR            9          1          3          6          9          1   

                                                      ...     return  \
date     2020-03-31 2020-06-30 2020-09-30 2021-01-31  ... 2020-06-30   
symbol                                                ...              
0O9.F             3          6          9          1  ...   1.377232   
0QF.F             3          6          9          1  ...   0.880384   
217A.F            3          6          9          1  ...   0.773163   
22UA.F            3          6          9          1  ...   0.159442   
2CRSI.PA          3          6          9          1  ...   0.702118   
...             ...        ...        ...        ...  ...        ...   
XIOR.BR           3          6          9          1  ...   0.230433   
YATRA.AS          3          6          9          1  ...  -0.076699   
YK6B.F            3          6          9          1  ...        NaN   
ZEL.NZ            3          6          9          1  ...  -0.063918   
ZEN.BR            3          6          9          1  ...   0.116041   

                                                                            \
date     2020-09-30 2021-01-31 2021-03-31 2021-06-30 2021-09-30 2022-01-31   
symbol                                                                       
0O9.F      0.080483   1.379888  -0.230569  -0.085763  -0.259548  -0.401502   
0QF.F      0.121367   1.177801  -0.179831   0.822632   0.673667  -0.547315   
217A.F    -0.127928   0.657025  -0.150561  -0.313028  -0.225427  -0.448966   
22UA.F    -0.006257   0.518609  -0.002897   1.111287   0.264526  -0.350037   
2CRSI.PA  -0.161769   0.490524   0.077559  -0.172468  -0.034221   0.093258   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR    0.156309  -0.074919  -0.106025   0.114661  -0.026894  -0.023805   
YATRA.AS  -0.100946   0.111111  -0.025000  -0.016464  -0.094402   0.000000   
YK6B.F          NaN        NaN   0.099440   0.150318  -0.070875  -0.085221   
ZEL.NZ     0.005140   0.105917  -0.058785  -0.045614   0.253676   0.055132   
ZEN.BR     0.033537   0.134234  -0.013043  -0.051057   0.058447   0.000000   

                                           
date     2022-03-31 2022-06-30 2022-09-30  
symbol                                     
0O9.F           0.0        0.0        0.0  
0QF.F           0.0        0.0        0.0  
217A.F          0.0        0.0        0.0  
22UA.F          0.0        0.0        0.0  
2CRSI.PA        0.0        0.0        0.0  
...             ...        ...        ...  
XIOR.BR         0.0        0.0        0.0  
YATRA.AS        0.0        0.0        0.0  
YK6B.F          0.0        0.0        0.0  
ZEL.NZ          0.0        0.0        0.0  
ZEN.BR          0.0        0.0        0.0  

[1407 rows x 4845 columns]

In [43]:
X_test = X_test.select_dtypes(exclude=["object", "datetime"])
assert len(X_train.columns) == len(X_test.columns)

In [44]:
model.predict(X_test)

array([ 0.00728833, -0.02220412, -0.01943101, ...,  0.00059691,
        0.0005502 ,  0.00066021], dtype=float32)